# Język Python - Wykład 7.

## Idiomy

### Uruchamianie i import

In [ ]:
def foo():
    pass
    
print('W module: ', __name__)

In [ ]:
def main():
    print('W module: ', __name__)
    
if __name__ == '__main__':
    main()

### Warunki logiczne

In [ ]:
# GOOD
name = 'Safe'
pets = ['Dog', 'Cat', 'Hamster']
owners = {'Safe': 'Cat', 'George': 'Dog'}

if name and pets and owners:
    print('We have pets!')

In [ ]:
# NOT SO GOOD
if name != '' and len(pets) > 0 and owners != {}:
    print('We have pets!')

### Przynależność

In [ ]:
# GOOD
name = 'Safe Hammad'
if 'H' in name:
    print('This name has an H in it!')

In [ ]:
# NOT SO GOOD
name = 'Safe Hammad'
if name.find('H') != -1:
    print('This name has an H in it!')

### Modyfikowalna wartość domyślna

In [ ]:
# VERY BAD
def f(arg=[]):
    arg += [1]
    return arg

In [ ]:
# ALSO BAD
def g(arg=None):
    arg = arg or []
    arg += [1]
    return arg

In [ ]:
# GOOD
def h(arg=None):
    if arg is None:
        arg = []
    arg += [1]
    return arg

In [ ]:
h()

### Łączenie list

In [ ]:
# GOOD
pets = ['Dog', 'Cat', 'Hamster']
ages = [5, 2, 3]
for pet, age in zip(pets, ages):
    print("{} is {} years old".format(pet, age))

In [ ]:
# NOT SO GOOD
pets = ['Dog', 'Cat', 'Hamster']
ages = [5, 2, 3]
i = 0
while i < len(pets):
    print(pets[i], 'is', ages[i], 'years old')
    i += 1

Ile ruchów trzeba wykonać, żeby włożyć żyrafę do lodówki?

### Iteracja z indeksem

In [ ]:
# GOOD
pets = ['Dog', 'Cat', 'Hamster']
for i, elem in enumerate(pets):
    print(i, elem)

In [ ]:
# NOT SO GOOD
pets = ['Dog', 'Cat', 'Hamster']
for i in range(len(pets)):
    print(i, pets[i])

### Składanie tekstu

In [ ]:
# GOOD
chars = ['S', 'a', 'f', 'e']
name = ''.join(chars)
print(name)

In [ ]:
# NOT SO GOOD
chars = ['S', 'a', 'f', 'e']
name = ''
for char in chars:
    name += char
print(name)

### EAFP vs LBYL
“It's Easier to Ask for Forgiveness than
Permission.”
“Look Before You Leap”

In [ ]:
# GOOD
d = {'x': '5'}
try:
    value = int(d['x'])
except (KeyError, TypeError, ValueError):
    value = None

In [ ]:
# NOT SO GOOD
d = {'x': '5'}
if 'x' in d and isinstance(d['x'], str) and d['x'].isdigit():
    value = int(d['x'])
else:
    value = None

### List comprehension

In [ ]:
# GOOD
data = [7, 20, 3, 15, 11]
result = [i * 3 for i in data if i > 10]
print(result)

In [ ]:
# NOT SO GOOD (MOST OF THE TIME)
data = [7, 20, 3, 15, 11]
result = []
for i in data:
    if i > 10:
        result.append(i * 3)
print(result)

A jeśli się da, to użyjmy generatora

### Upraszczanie wyboru

In [ ]:
number_type = '?'
if number_type == 'floating_point':
    converter = float
elif number_type == 'integer':
    converter = int
else:
    converter = complex
converter("12")

In [ ]:
number_type = '?'
converters = {
    'floating_point' : float,
    'integer' : int,
}
converter = converters.get(number_type, complex)
converter("12")

In [ ]:
from collections import defaultdict

number_type = '?'
converters = defaultdict(lambda: complex, {
    'floating_point' : float,
    'integer' : int,
})
converters[number_type]("12")

## Wzorce projektowe

### Wbudowane:
  - iterator
  - dekorator

### Zasoby:
    https://github.com/faif/python-patterns

### Singleton

Definicja:
 -  jeden obiekt danej klasy

Zastosowania:
 * potrzeba tylko jednego obiektu (np. połączenie do bazy danych, logger)
 * kontrola współbieżnego dostępu do współdzielonego zasobu
 * singleton tylko-do-odczytu dla globalnego stanu (np. strefa czasowa, język, stałe fizyczne)
 
Bywa nadużywany i stosowany w zastępstwie zmiennych globalnych.

In [ ]:
class Singleton():
    
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super().__new__(cls)
        return cls.instance

In [ ]:
o1 = Singleton()
o2 = Singleton()

In [ ]:
o1.x = 2

In [ ]:
print(o2.x)

In [ ]:
id(o1),id(o2)

In [ ]:
o1 is o2, o1 == o2

### Dzielenie stanu

In [ ]:
class Borg():
    
    _shared_state = {}
    
    def __new__(cls, *args, **kwargs):
        obj = super().__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj

In [ ]:
b1 = Borg()
b2 = Borg()

In [ ]:
b1.x = 3
b2.x

In [ ]:
id(b1), id(b2)

In [ ]:
b1 is b2

https://chromium.googlesource.com/external/googleappengine/python/+/master/google/pyglib/singleton.py

### MVC

Definicja:
 - model  - DANE i LOGIKA BIZNESOWA (nie zależy od widoku i kontrolera)
 - widok (view) - PREZENTACJA (zależy od modelu)
 - kontroler (controller) - ŁĄCZY WIDOK i MODEL (zależy od modelu i widoku)

In [ ]:
class Model():

    products = {
        'milk': {'price': 1.50, 'quantity': 10},
        'eggs': {'price': 0.20, 'quantity': 100},
        'cheese': {'price': 2.00, 'quantity': 10}
    }

In [ ]:
class View():

    def product_list(self, product_list):
        print('PRODUCT LIST:')
        for product in product_list:
            print(product)
        print('')

    def product_information(self, product, product_info):
        print('PRODUCT INFORMATION:')
        print('Name: %s, Price: %.2f, Quantity: %d\n' %
              (product.title(), product_info.get('price', 0),
               product_info.get('quantity', 0)))

    def product_not_found(self, product):
        print('That product "%s" does not exist in the records' % product)

In [ ]:
class Controller():

    def __init__(self, model, view):
        self.model = model
        self.view = view

    def get_product_list(self):
        product_list = self.model.products.keys()
        self.view.product_list(product_list)

    def get_product_information(self, product):
        product_info = self.model.products.get(product, None)
        if product_info is not None:
            self.view.product_information(product, product_info)
        else:
            self.view.product_not_found(product)

In [ ]:
controller = Controller(Model(),View())
controller.get_product_list()
controller.get_product_information('cheese')
controller.get_product_information('eggs')
controller.get_product_information('milk')
controller.get_product_information('arepas')

In [ ]:
class ViewOther():

    def product_list(self, product_list):
        print('PRODUCT LIST:')
        for product in product_list:
            print(" - ", product)
        print('')

In [ ]:
controller = Controller(Model(), ViewOther())
controller.get_product_list()

### Factory method pattern

In [ ]:
class A():
    def __init__(self):
        self.a = "Hello"

class B():
    def __init__(self):
        self.a = " World"
    
myfactory = {
    "greeting" : A,
    "subject" : B,
}

In [ ]:
myfactory["greeting"]().a

### Strategia

In [ ]:
def bisection(line):
    return 5.5, 6.6

def conjugate_gradient(line):
    return 3.3, 4.4

def test():
    solver = conjugate_gradient
    print(solver((5.5, 5.5)))
    solver = bisection
    print(solver((5.5, 5.5)))

test() 

### Observer

In [ ]:
class Point():
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def scale(self, n):
        self.x = n * self.x
        self.y = n * self.y
        
def notify(f):
    def g(self, n):
        print("executed", n)
        return f(self, n)
    return g

Point.scale = notify(Point.scale)
p = Point(2.0, 3.0)
p.scale(2.5)

In [ ]:
def notify(f):
    def g(self, n):
        print("executed", n)
        return f(self, n)
    return g

class Point():
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @notify
    def scale(self, n):
        self.x = n * self.x
        self.y = n * self.y
        

p = Point(2.0, 3.0)
p.scale(2.5)

## Wątki

https://docs.python.org/3/library/threading.html

A programmer had a problem.

"I know! I will solve it with threads!" - he thought to himself.

has Now problems. two he

### Tworzenie i uruchamianie wątku

In [ ]:
import threading

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            if i % 1e4 == 0:
                print(i)
                
t1 = Thread1()
t2 = Thread1()

t1.start()
t2.start()
print("Finished")

In [ ]:
p = threading.Thread(target=print, args=('bob', 'john'), kwargs={'sep': '\t'})
p.start()

A gdyby tak:

In [ ]:
@threaded
def f():
    pass

f()

### Synchronizacja wątków

#### Lock

In [ ]:
import threading
import time

queue = list(range(10))
lock = threading.Lock()

class Consumer(threading.Thread):
    def run(self):
        running = True
        while running:
            lock.acquire()
            if not queue:
                running = False
            else:
                elem = queue.pop()
            lock.release()
            time.sleep(2) # do something with elem
            lock.acquire()
            print(self.name, elem)
            lock.release()
            
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()
t1.join()
t2.join()
print("Finished")

In [ ]:
import threading
import time

queue = list(range(10))
lock = threading.Lock()

class Consumer(threading.Thread):
    def run(self):
        while True:
            with lock:
                if not queue:
                    break
                else:
                    elem = queue.pop()
            time.sleep(3) # do something with elem
            with lock:
                print(self.name, elem)
            
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()
t1.join()
t2.join()
print("Finished")

`Lock`:
* może zostać zwolniony przez **dowolny wątek**
* próba ponownego zajęcia przez ten sam wątek blokuje go *ad infinitum*
* nie można zwolnić, jeżeli nie został zajęty
* można spróbować zająć w trybie nieblokującym
* nie jest automatycznie zwalniany

Polecam `RLock`:
* może zostać zwolniony tylko przez wątek, który go posiada
* można go zajmować wielokrotnie, ale należy zwolnić tyle samo razy
* jest zwalniany automatycznie po zakończeniu wątku

A gdyby tak:

In [ ]:
@synchronized
def f():
    pass

In [ ]:
type(threading.Lock())

In [ ]:
import threading
import time

lock = threading.RLock()

class Thread1(threading.Thread):
    def run(self, *args):
        print(lock.acquire(False))
#         print(lock.acquire(timeout=3))  # alt
        time.sleep(1)
        lock.release()
        
                
Thread1().start()
# time.sleep(2)
Thread1().start()

#### Condition

In [ ]:
import threading
import time

cv = threading.Condition()

l = []

class Consumer(threading.Thread):
    def run(self):
        '''Consume one item'''
        with cv:
            while True:
                while not l:
                    cv.wait()
                print(l.pop(0))

class Producer(threading.Thread):
    def run(self):
        '''Produce one item'''
        global l
        for i in range(10):
            with cv:
                l += [i]
                cv.notify()
            time.sleep(1)

Consumer().start()
time.sleep(2)
Producer().start()

In [ ]:
print(l)

#### Inne

* `Semaphore`
* `BoundedSemaphore`
* `Event`
* `Timer`
* `Barrier`

### Wydajność

In [ ]:
import threading
import time

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.start()
t2.start()
print("Finished")

t1.join()
t2.join()

finish = time.time()
print(finish - start)

In [ ]:
import threading
import time

class Thread1(threading.Thread):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.run()
t2.run()
print("Finished")

finish = time.time()
print(finish - start)

## Procesy

https://docs.python.org/3/library/multiprocessing.html

### Wydajność

In [ ]:
import multiprocessing
import time

class Thread1(multiprocessing.Process):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.run()
t2.run()
print("Finished")

finish = time.time()
print(finish - start)

In [ ]:
import multiprocessing
import time

class Thread1(multiprocessing.Process):
    def run(self):
        for i in range(int(1e6)):
            a = i**2
            b = i**3
            c = i**4
                
t1 = Thread1()
t2 = Thread1()

start = time.time()

t1.start()
t2.start()
print("Finished")

t1.join()
t2.join()
finish = time.time()
print(finish - start)

Procesy są cięższe i mniej wygodne w użyciu, ale omijają GIL.

In [ ]:
import multiprocessing
import time

queue = list(range(10))
lock = multiprocessing.Lock()

class Consumer(multiprocessing.Process):
    def run(self):
        while True:
            with lock:
                if not queue:
                    break
                else:
                    elem = queue.pop()
            time.sleep(2) # do something with elem
            with lock:
                print(self.pid, elem)
                        
t1 = Consumer()
t2 = Consumer()
t1.start()
t2.start()

Przydatne klasy:
- Lock
- Queue
- JoinableQueue
- Pipe
- Value
- Array

## Wyrażenia regularne

https://docs.python.org/3/library/re.html

In [ ]:
"ABC123".isupper()

In [ ]:
import regex  # re
print(regex.fullmatch('[A-Z]+', "ABC123"))
print(regex.fullmatch('[A-Z]+', "ABC"))

In [ ]:
s = '\\'
print(regex.fullmatch('\\\\', s))

In [ ]:
print(s)

In [ ]:
print('\n')
print(r'\n')

Najważniejsze funkcje:
* match / fullmatch
* search
* findall / finditer
* sub
* split

In [ ]:
match = regex.match(r'[a-z]+(.[a-z]+)+', "agh.edu.pl")
print(match)

In [ ]:
regex.match(r'[a-z]+(.[a-z]+)+', "agh")

In [ ]:
match.groups()

In [ ]:
match = regex.match(r'[a-z]+(\.[a-z]+)+', "Website: agh.edu.pl")
print(match)
match = regex.search(r'[a-z]+(\.[a-z]+)+', "Website: agh.edu.pl")
print(match)

In [ ]:
match = regex.match(r'[a-z]+(\.[a-z]+)+', "koło.pl")
print(match)
match = regex.match(r'\p{Ll}+(\.\w+)+', "koło.pl")
print(match)

Wyrażenie regularne może zawierać:
* znaki
* klasy znaków: `.`, `[a-z]`, `\w`
* operatory powtórzenia: `*`, `+`, `?`, `*?`, `+?`, `??`, `{m}`, `{m,n}`, `{m,n}?`
* operator alternatywy: `|`
* początek/koniec napisu: `^`/`$`
* grupy: `(...)`
* kontekst: `(?=...)`,`(?!...)`, `(?<=...)`, `(?<!...)`
* grupy nazwane: `(?P<grupa>...)`
* grupy nieprzechwytujące: `(?:...)`
* referencje wsteczne: `(?P=grupa)`, `\1`, `\99`
* ...

In [ ]:
regex.sub(r"([a-z])", r"_\1", "abcd")

In [ ]:
regex.split("c(?=d)", "abcdefcgh")

regex.I - case insensitive

regex.M - multiline ($ może oznaczać koniec linii, a nie tylko koniec łańcucha)

regex.S - kropka może oznaczać też koniec linii

In [ ]:
regex.match('\w+', "犬いぬ")

\w - word character

\s - whitespace

\d - digit

\b - word boundary (albo backspace)

In [ ]:
r'\p{Ll}'

Klasy znaków Unicode: https://www.fileformat.info/info/unicode/category/index.htm

## Django

https://www.djangoproject.com/

Fragmenty kodu za: https://matthewdaly.co.uk/blog/2013/12/28/django-blog-tutorial-the-next-generation-part-1/

** The Web framework for perfectionists with deadlines **

* DRY - Don't Repeat Yourself. "Every piece of knowledge must have a single, unambiguous, authoritative representation within a system." http://c2.com/cgi/wiki?DontRepeatYourself
* MVC vs MVT - django często określane jest mianem MVT - Model View Template
* ORM - Object Relational Mapper (modele django)
* Routing URL oparty na wyrażeniach regularnych
* System szablonów 
* Cache'ing 
* Lokalizacja i tłumaczenia
* Automatyczny panel admina

### Aplikacje

Projekt składa się z aplikacji.

Projekty i aplikacje łączy zależność 'wiele do wielu'.

Plik `settings.py` określa aplikacje zainstalowane w projekcie.

### ORM (Object-Relational Mapping)

Plik `models.py` w katalogu aplikacji.

In [ ]:
from django.db import models

# Create your models here.

class Post(models.Model):
    title = models.CharField(max_length=200)
    pub_date = models.DateTimeField()
    text = models.TextField()

In [ ]:
from django.db import models
[cls for cls in dir(models) if cls.endswith("Field")]

### Routing URL

In [ ]:
#plik urls.py projektu

from django.conf.urls import include, url

from django.contrib import admin

urlpatterns = [
    url(r'^admin/', include(admin.site.urls)),
    url(r'^.*$', include('blogengine.urls')),
]

In [ ]:
# plik urls.py aplikacji

from django.conf.urls import patterns, url
from django.views.generic import ListView

from blogengine.models import Post
from blogengine import views

urlpatterns = [
    url('^$', ListView.as_view(model=Post,)),
    url('^(?P<id>\d+)$', views.post_view, name='postview')
]

### Widoki

In [ ]:
# w pliku views.py aplikacji

@login_required(login_url='/user/login/')
def post_view(request, id):
    post = Post.objects.filter(id=id)[0]
    context = {'post': post}
    return render(request, 'views/post.html', context)

### Szablony

Alternatywnie: